# Getting Started

At minimum it is expected that your development environment includes:

- Unix/Linux shell
- Make (typically already installed)
- Docker
- Go (can be installed using vs code)
- Python 3 (can be installed using vs code)

Most other tools are installed automatically when invoking `make` targets. These tools will be installed in the `bin` subdirectory in the root of this repo aka `hello-world-api/bin`.





## Prerequisites

In the repo root directory it is expected that you have a private ssh key which can be used to fetch private dependencies from GitHub over ssh `hello-world-api/.ssh/id_rsa`

If this file does not exist you may not be able to build code that relies on private dependencies. The `make` scripts will try to create this for you by copying your ssh key from `$HOME/.ssh/id_rsa`

# Jupyter Notebooks

It is recommended that you use VS Code when working with this repo and install the Jupyter Notebooks extension to run these notebooks.

In [ ]:
# Run this before any other cells
import subprocess
import os
import signal

def run_command(cmd, capture_output=True):
    print(cmd)
    return subprocess.run(cmd, shell=True, capture_output=capture_output)

def cleanup(cmdstdout):
    for pid in cmdstdout.stdout.decode('utf-8').split('\n'):
        if pid:
            print(f'Interrupting process {pid}')
            try:
                os.kill(int(pid), signal.SIGINT) #or signal.SIGKILL
            except:
                pass
            finally:
                pass

## Development Environment

The below command will create a local kubernetes cluster for you using k3d. This will be your development environment.

In [ ]:
%%capture
# The above line discards shell output in the notebook so that it is not committed to the repo. Feel free to comment it out to view the output.
# The below command will create a local kubernetes cluster for you using k3d. 
# This will be your development environment.
! cd .. && make bootstrap

### Database

To connect to a local database, you can forward a port from the cluster to localhost

In [ ]:

run_command("cd .. && kubectl port-forward deployment/postgres 5432:5432 &", capture_output=False)

Use the below command to disable port-forwarding

In [ ]:
cleanup(run_command("ps -a -x | grep 'kubectl port-forward deployment/postgres 5432:5432' | grep -E -o '^\s?[0-9]+\s'"))

### Deploying Locally

Once the environment is bootstrapped, you should be able to deploy the service using skaffold.


#### Run Mode

If you just want to run the stack locally and you have no intention to debug or actively change code, use `make run` or `make run-skaffold`

This will expose the services on the route `http://localhost:8081/hello-world-api/api/1`

In [ ]:
%%capture 
# The above line discards shell output in the notebook so that it is not committed to the repo. Feel free to comment it out to view the output.
! cd .. && make run

#### Dev Mode

If you intend to actively change code, and want hot reloading use `make dev`. This will deploy the same code but with hot reloading. As you edit and save your code, skaffold will replace the relevant docker layers in the running container. (Warning: This does not always work and your container may not restart).

While in dev mode, skaffold will automatically forward ports to localhost for the relevant services so that you can immediately send requests to the services.

In [ ]:
%%capture
run_command("cd .. && make dev &", capture_output=False)

Use the below command to kill the skaffold dev process

In [ ]:
cleanup(run_command("ps -a -x | grep 'skaffold dev' | grep -E -o '^\s?[0-9]+\s'"))
run_command("cd .. && make clean-skaffold")

#### Debug Mode

If you need to debug a service while it is running, you can use `make debug`. This will deploy the same code with a debug server exposed. See `infra/k8s/overlays/debug` to find the exposed port and yaml definitions.

In the case of the Go app, this will build a different image `go/Dockerfile.dbg` with delve built in and expose port 2345.

While in debug mode, skaffold will automatically forward ports to localhost for the relevant services and the debug port so that you can immediately send requests to the services.

In [ ]:
%%capture
run_command("cd .. && make debug &", capture_output=False)

Use the below command to kill the skaffold dev process

In [ ]:
cleanup(run_command("ps -a -x | grep 'skaffold debug' | grep -E -o '^\s?[0-9]+\s'"))
run_command("cd .. && make clean-skaffold")

# Directories

## notebooks/

This directory stores interactive notebooks for documentation and other useful purposes. Some use cases

- Onboarding
- System Architecture Diagrams
- Runbooks for testing or incidents
- Gherkin Feature Specification

## bin/

Executables for the local development environment are installed in the `bin` directory. This directory should not be managed manually as the makefiles may overwrite it and it is deliberately ignored in this repo.

## config/

Configuration files for services are defined in the `config` directory.

## gen/

This directory contains generated documentation for the API including a swagger specification generated from the protocol buffer specifications and markdown documents. The files in this directory should not be manually altered as they will be overwritten if docs are regenerated.

## go/

All Go source code lives in the `go` directory including dockerfiles for Go artifacts

## infra/

Most of the infrastructure related scripts and configuration lives in the `infra` directory. The exception is `skaffold.yaml` which is in the root directory.

## makefiles/

This directory defines makefiles and targets for CI/CD orchestration, primarily in a local dev environment.

## proto/

This is the source of truth for our API specifications. All APIs are defined using protocol buffers and live in the `proto` directory. Any new APIs should be defined in protobuf *first*. Code generators will automatically pull the spec from this directory.


## test/k6

This directory contains source code for K6 tests. These tests are intended to run as smoke, load, stress and/or soak tests. These are blackbox tests and are agnostic of the backend application stack. They can be written as HTTP tests or gRPC tests.

# hello-world-api Domains

There is one API documented in the `proto` directory using Protocol Buffers.

- Hello World API